In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.freesurfer as fsr
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()


In [2]:
# Set variables
user = expanduser('~')
if user == '/Users/lucindasisk':
    home = join(user, 'Desktop/Milgram/candlab')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    workflow_dir = join(home, 'analyses/shapes/dwi/preproc_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/preproc_data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    workflow_dir = join(home, 'analyses/shapes/dwi/preproc_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/preproc_data')
    
# Read in subject subject_list
# subject_list = read_csv(
#     home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ').values.tolist()

# Manual subject list
subject_list = ['sub-A200', 'sub-A201']


In [3]:
# 9/22/19: change so that T1 is registered to B0 per https://mrtrix.readthedocs.io/en/latest/quantitative_structural_connectivity/act.html

In [4]:
# Create preprocessing Workflow

# set default FreeSurfer subjects dir
fsr.FSCommand.set_default_subjects_dir(raw_dir)

# Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory=data_dir,
                         substitutions=[('_subject_id_', '')]),
                name='datasink')

# Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles
template = dict(t1=join(raw_dir, '{subject_id}/ses-shapesV1/anat/{subject_id}_ses-shapesV1_T1w.nii.gz'),
                dti=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.nii.gz'),
                bval=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bvec'),
                fmappa=join(
                    raw_dir, '{subject_id}/ses-shapesV1/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-PA_epi.nii.gz'),
                fmapap=join(
                    raw_dir, '{subject_id}/ses-shapesV1/fmap/{subject_id}_ses-shapesV1_acq-dwi_dir-AP_epi.nii.gz'),
                aps=join(raw_dir, 'shapes_acqparams.txt'),
                index=join(raw_dir, 'shapes_index.txt'),
                mni=join(home, 'atlases/MNI152_T1_2mm_brain.nii.gz')
                )

sf = Node(SelectFiles(template,
                      base_directory=home),
          name='sf')



In [5]:
# Merge AP/PA encoding direction fieldmaps
def create_merged_files(ap, pa):
    from nipype.interfaces import fsl
    from os.path import abspath
    merge = fsl.Merge(in_files=[ap, pa],
                      dimension='t', output_type='NIFTI_GZ', merged_file='AP_PA_merged.nii.gz').run()
    merged_file = abspath('AP_PA_merged.nii.gz')
    return merged_file


create_merge = Node(Function(input_names=['ap', 'pa'],
                             output_names=['merged_file'],
                             function=create_merged_files),
                    name='create_merge')

In [6]:
# Resample T1w to same voxel dimensions as DTI to avoid data interpolation (1.714286 x 1.714286 x 1.700001) .
resampt1 = Node(fsr.Resample(voxel_size=(1.714286, 1.714286, 1.700001)),
                name='resampT1')

# Drop bottom slice (S/I) to create even # of slices
drop = Node(fsl.ExtractROI(x_min=0, x_size=140,
                           y_min=0, y_size=140,
                           z_min=1, z_size=80, output_type='NIFTI_GZ'),
            name='drop')

# drop bottom slice of DTI file
drop2 = drop.clone(name='drop2')

# Denoise DWI data susing local PCA correction - mrTrix3
denoise = Node(mtx.DWIDenoise(out_file='denoised.nii.gz'),
               name='denoise')

# Steps added 7/17 per Jiook's reccomendations
# Gibbs ringing removal
gibbs = Node(mtx.MRDeGibbs(out_file='denoised_gibbs.nii.gz'),
             name='gibbs')

# DWI bias file correction using ANTS N4
bias = Node(mtx.DWIBiasCorrect(use_ants=True,
                               out_file='denoised_gibbs_bias.nii.gz'),
            name='bias')

###########################

# Run topup on merged files from pe1 and pe0
topup = Node(fsl.TOPUP(config='b02b0.cnf',
                       out_corrected='ap_pa_topup.nii.gz', output_type='NIFTI_GZ'),
             name='topup')

# Select b0 image for registration
fslroi = Node(fsl.ExtractROI(t_min=0,
                             t_size=1,
                             roi_file='b0_img.nii.gz', output_type='NIFTI_GZ'),
              name='fslroi')

# Reorient topup b0 image to std
reorient1 = Node(fsl.Reorient2Std(output_type='NIFTI_GZ'),
                 name='reorient1')

# Register T1 to MNI - rigid 2D transformation
register1 = Node(fsl.FLIRT(out_matrix_file='b0toT1_reorient_reg.mat',
                           rigid2D=True,
                           output_type='NIFTI_GZ',
                          no_resample=True),
                 name='register1')

# apply topup from merged file to rest of pe0 scan
apptop = Node(fsl.ApplyTOPUP(method='jac',
                             in_index=[2], 
                             output_type='NIFTI_GZ',
                            out_corrected = 'preprocessed_dwi.nii.gz'),
              name='apptop')

# Skullstrip the T1w image
stripT1 = Node(fsl.BET(mask=True, output_type='NIFTI_GZ'),
               name='stripT1')

#Eddy_CUDA Node
# FSL Eddy correction to remove eddy current distortion

eddy = Node(fsl.Eddy(is_shelled=True,
                     interp='trilinear',
                     method='jac',
                     output_type='NIFTI_GZ',
                     residuals=True,
                     use_cuda=True,
                     cnr_maps=True,
                     repol=True),
            name='eddy')



In [7]:

preproc_flow = Workflow(name='preproc_flow')
preproc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                      # Select AP and PA encoded fieldmaps; merge niftis
                      (sf, create_merge, [('fmapap', 'ap'),
                                          ('fmappa', 'pa')]),
                      # Drop bottom slice of nifi (had odd # slices)
                      (create_merge, drop, [('merged_file', 'in_file')]),
                      # Run topop across merged niftis
                      (drop, topup, [('roi_file', 'in_file')]),
                      (sf, topup, [('aps', 'encoding_file')]),
                      (topup, datasink, [
                       ('out_corrected', '1_Check_Unwarped.@par')]),
                      # Extract b0 image from nifti with topup applied
                      (topup, fslroi, [('out_corrected', 'in_file')]),
                      # Resample T1w to same voxel dimensions as DTI
                      (sf, resampt1, [('t1', 'in_file')]),
                      #Register T1 to b0 brain
                      (resampt1, register1, [('resampled_file', 'in_file')]),
                      (fslroi, register1, [('roi_file', 'reference')]),
                      #skullstrip T1
                      (register1, stripT1, [('out_file', 'in_file')]),
                      # Save stripped anat and mask
                      (stripT1, datasink, [('mask_file', '1_Check_Unwarped.@par.@par.@par.@par.@par.@par'),
                                           ('mask_file', '2_Transfer')]),
                      (register1, datasink, [('out_file', '1_Check_Unwarped.@par.@par.@par.@par.@par.@par.@par'),
                                             ('out_file', '2_Transfer.@par')]),
                      # Drop bottom slice from DTI nifti
                      (sf, drop2, [('dti', 'in_file')]),
                      # Local PCA to denoise DTI data
                      (drop2, denoise, [('roi_file', 'in_file')]),
                      (denoise, datasink, [
                       ('out_file', '1_Check_Unwarped.@par.@par.@par')]),
                      # Gibbs ringing removal
                      (drop2, gibbs, [('roi_file', 'in_file')]),
                      # Perform DWI bias field correction
                      (gibbs, bias, [('out_file', 'in_file')]),
                      (sf, bias, [('bvec', 'in_bvec')]),
                      (sf, bias, [('bval', 'in_bval')]),
                      # Apply topup to bias corrected DTI data
                      (topup, apptop, [('out_fieldcoef', 'in_topup_fieldcoef'),
                                      ('out_movpar','in_topup_movpar')]),
                      (bias, apptop, [('out_file', 'in_files')]),
                      (sf, apptop, [('aps', 'encoding_file')]),
                      (apptop, datasink, [
                          ('out_corrected', '1_Check_Unwarped.@par.@par.@par.@par.@par'),
                          ('out_corrected', '2_Transfer.@par.@par')]),
                      (sf, datasink, [('bval', '2_Transfer.@par.@par.@par'),
                                      ('bvec', '2_Transfer.@par.@par.@par.@par')])
                      ])
preproc_flow.base_dir = workflow_dir
preproc_flow.write_graph(graph2use='flat')
preproc = preproc_flow.run('MultiProc', plugin_args={'n_procs': 4})

190923-19:21:18,947 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): No edge data
190923-19:21:18,948 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190923-19:21:18,949 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): No edge data
190923-19:21:18,951 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): new edge data: {'connect': [('fmapap', 'ap'), ('fmappa', 'pa')]}
190923-19:21:18,951 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.drop): No edge data
190923-19:21:18,952 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.drop): new edge data: {'connect': [('merged_file', 'in_file')]}
190923-19:21:18,952 nipype.workflow DEBUG:
	 (preproc_flow.drop, preproc_flow.topup): No edge data
190923-19:21:18,953 nipype.workflow DEBUG:
	 (preproc_flow.drop, preproc_flow.topup): new edge data: {'connect': [('roi_file', 'in_file

190923-19:21:20,236 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.80, cwd=/Users/lucindasisk/Dropbox/Github/Diffusion)
190923-19:21:20,243 nipype.workflow DEBUG:
	 Creating flat graph for workflow: preproc_flow
190923-19:21:20,250 nipype.workflow DEBUG:
	 expanding workflow: preproc_flow
190923-19:21:20,251 nipype.workflow DEBUG:
	 processing node: preproc_flow.infosource
190923-19:21:20,252 nipype.workflow DEBUG:
	 processing node: preproc_flow.sf
190923-19:21:20,253 nipype.workflow DEBUG:
	 processing node: preproc_flow.drop2
190923-19:21:20,253 nipype.workflow DEBUG:
	 processing node: preproc_flow.gibbs
190923-19:21:20,254 nipype.workflow DEBUG:
	 processing node: preproc_flow.bias
190923-19:21:20,255 nipype.workflow DEBUG:
	 processing node: preproc_flow.denoise
190923-19:21:20,256 nipype.workflow DEBUG:
	 processing node: preproc_flow.resampT1
190923-19:21:20,257 nipype.workflow DEBUG:
	 processing node: preproc_flow.create_merge
190923-19:21:20,257 nipype.w

	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/anat/sub-A200_ses-shapesV1_T1w.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bvec;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/fmap/sub-A200_ses-shapesV1_acq-dwi_dir-PA_epi.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/fmap/sub-A200_ses-shapesV1_acq-dwi_dir-AP_epi.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/shapes_acqparams.txt;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/shapes_index.txt;/Us

	 [Node] Setting-up "preproc_flow.resampT1" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1".190923-19:21:22,879 nipype.workflow DEBUG:
	 output: fmapap

190923-19:21:22,880 nipype.workflow DEBUG:
	 [Node] create_merge - setting input ap = /Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/fmap/sub-A200_ses-shapesV1_acq-dwi_dir-AP_epi.nii.gz
190923-19:21:22,881 nipype.workflow DEBUG:
	 input: pa
190923-19:21:22,883 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf/result_sf.pklz
190923-19:21:22,884 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/sf/result_sf.pklz
190923-19:21:22,891 nipype.workflow DEBUG:
	 output: fmappa
190923-19:21:22,892 nipype.workflow DEBUG:


190923-19:21:24,803 nipype.workflow DEBUG:
	 [MultiProc] Submitted task preproc_flow.gibbs (taskid=3).
190923-19:21:24,804 nipype.workflow DEBUG:
	 Allocating preproc_flow.denoise ID=4 (0.20GB, 1 threads). Free: 28.20GB, 1 threads.
190923-19:21:24,805 nipype.workflow DEBUG:
	 Setting node inputs
190923-19:21:24,806 nipype.workflow DEBUG:
	 input: in_file
190923-19:21:24,807 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/drop2/result_drop2.pklz
190923-19:21:24,808 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/drop2/result_drop2.pklz
190923-19:21:24,807 nipype.workflow INFO:
	 [Node] Setting-up "preproc_flow.gibbs" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/gibbs".
190923-19:21:24,822 nipype.workflow WARNING:

	 some file does not exist. hence trait cannot be set190923-19:21:24,946 nipype.workflow DEBUG:
	 output: merged_file

190923-19:21:24,952 nipype.workflow DEBUG:
	 [Node] drop - setting input in_file = /gpfs/milgram/project/gee_dylan/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/create_merge/AP_PA_merged.nii.gz
190923-19:21:24,952 nipype.workflow DEBUG:
	 Aggregate: True
190923-19:21:24,965 nipype.workflow DEBUG:
	 some file does not exist. hence trait cannot be set
190923-19:21:24,972 nipype.workflow DEBUG:
	 Aggregate: True
190923-19:21:25,23 nipype.workflow DEBUG:
	 some file does not exist. hence trait cannot be set
190923-19:21:25,29 nipype.workflow DEBUG:
	 Aggregate: True
190923-19:21:26,672 nipype.workflow DEBUG:
	 Progress: 14 jobs, 3/4/0 (done/running/ready), 4/7 (pending_tasks/waiting).
190923-19:21:26,673 nipype.workflow DEBUG:
	 Clearing 2 from queue
190923-19:21:26,685 nipype.workflow DEBUG:
	 some file does not exist. hence trait cannot b

RuntimeError: Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/plugins/multiproc.py", line 69, in run_node
    result['result'] = node.run(updatehash=updatehash)
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 410, in run
    cached, updated = self.is_cached()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 296, in is_cached
    hashed_inputs, hashvalue = self._get_hashval()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 493, in _get_hashval
    self._get_inputs()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/pipeline/engine/nodes.py", line 520, in _get_inputs
    outputs = loadpkl(results_file).outputs
  File "/usr/local/anaconda3/lib/python3.6/site-packages/nipype/utils/filemanip.py", line 668, in loadpkl
    raise e
  File "/usr/local/anaconda3/lib/python3.6/site-p

190923-19:21:47,927 nipype.workflow DEBUG:
	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1/sub-A200_ses-shapesV1_T1w_resample.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/anat/sub-A200_ses-shapesV1_T1w.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1/_0xfc386246f9dc7118ee804d8f066652c2_unfinished.json;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1/command.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1/_inputs.pklz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_workflow/preproc_flow/_subject_id_sub-A200/resampT1/_node.pklz
190923-19:21:47,935 nipype.workflow DEBUG:
	 Needed dirs: /Users/l